This notebook calculates statistics on how well matching with different metrics preserves different attributes.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from copy import deepcopy
from os.path import join as oj

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics
import sklearn.decomposition
import sklearn.manifold

from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from tqdm import tqdm
import sys
sys.path.append('..')
sys.path.append('../projection_manipulation')
import transects

import data
import util
from matching import *
from config import *

df = data.load_all_labs()
df = df.set_index('fname_id')

# get fnames
fname_nps = [f for f in sorted(os.listdir(DIR_GEN)) if 'npy' in f] # these start at 00001
fname_ids = np.array([f[:-4] for f in fname_nps])

In [3]:
dists_match_names = ['facial', 'vgg', 'gan', 'gan_constrained']
d = df[df['count_with_this_id'] > 1]
d = df
d = d[[k for k in d.keys()
       if not ('md5' in k or 'file' in k or 'idx' in k or 'fname' in k or 'prob' in k or 'count_with' in k)]] # filter some keys we don't really care about

N_IMS = 1000
suffs = ['', '_diff']
errs_keys = ['err_top1', 'err_top5', 'err_top10']
numerical_keys = ['yaw', 'pitch', 'roll', 'background_mean', 'background_std', 'quality']
attr_keys = [kk for kk in d.keys() if not 'scores' in kk and not kk in numerical_keys]
all_keys = errs_keys + attr_keys + numerical_keys \
        + [kk + '_diff' for kk in attr_keys + numerical_keys]

# load and analyze the matching metrics

In [11]:
r = pd.read_pickle(oj(DIR_PROCESSED, 'matches_top10_stats.pkl')).round(3)
for k in all_keys:
    if not k in numerical_keys \
    and not k.replace('_diff', '') in numerical_keys \
    and not 'black_or_white' in k:
        r[k] *= 100
        r[k + '_std'] *= 100
r['quality'] *= 100
# for k in r.keys():
#     if r.loc['gan', k] > r.loc['facial', k]:
#         print(k, f"{r.loc['gan', k] - r.loc['facial', k]:0.1f}")
# r #.round(3).style.background_gradient()
r[[k for k in r.keys() if '_diff' not in k]]

,err_top1,err_top5,err_top10,id,gender,hair-length,facial-hair,makeup,skin-color,age,...,race_pred_std,race4_pred_std,gender_pred_std,age_pred_std,yaw_std,pitch_std,roll_std,background_mean_std,background_std_std,quality_std
facial,7.8,4.7,4.5,27.4,1.0,13.0,7.1,25.8,8.4,11.8,...,18.2,13.8,16.9,31.7,7.913,3.877,1.492,17.949,8.167,0.029
vgg,0.2,0.0,0.0,88.4,22.1,17.7,12.7,36.1,5.4,22.8,...,28.7,26.6,19.9,29.3,4.459,3.091,1.109,8.766,9.827,0.033
gan,80.4,72.4,67.6,94.9,16.0,14.8,16.4,30.8,7.1,24.2,...,29.7,29.6,23.0,33.3,3.167,3.028,1.145,11.242,5.610,0.039
gan_constrained,36.0,16.6,12.3,55.8,0.9,13.0,7.4,27.2,8.0,13.1,...,21.4,17.1,16.3,31.4,5.465,3.533,1.311,16.008,7.458,0.030


In [12]:
r.index = ['Facial-rec dist', 'VGG dist', 'GAN dist', 'Combined']
id_attributes = ['err_top1', 'gender', 'race_pred']
qual_attributes = ['blurry', 'quality']
image_attributes = ['background_mean', 'yaw', 'pitch', 'roll']
id_correlated_attributes = ['mustache', 'eyeglasses', 'bangs', 'wearing_hat']
# attrs = id_attributes + qual_attributes + image_attributes + id_correlated_attributes
attrs = image_attributes + id_attributes

r2 = deepcopy(r).round(1)

# add in stddev
n = r2.shape[0]
for k in r2.keys():
    if not 'std' in k:
        r2[k] = [str(r2[k].values[i]) + ' $\pm$ ' +  str(np.round(r2[k + '_std'].values[i] / np.sqrt(n), 0))
                 for i in range(n)]

        
rename = {
    'err_top1': 'ID (top1)',
    'race_pred': 'Race',
    'background_mean': 'Background Mean',
}
for k in attrs:
    if not k in rename:
        rename[k] = k.capitalize()        
r3 = r2[attrs].rename(columns=rename)

In [13]:
s = r3.loc[['GAN dist', 'Facial-rec dist', 'Combined']].to_latex().replace('textbackslash pm', 'pm').replace('\$', '$')
s = s.replace('llllllll', 'lllll|lll') # add in dividing line
print(s)
# r3

\begin{tabular}{lllll|lll}
\toprule
{} & Background Mean &             Yaw &          Pitch &           Roll &        ID (top1) &           Gender &             Race \\
\midrule
GAN dist        &  33.2 $\pm$ 6.0 &   6.6 $\pm$ 2.0 &  5.6 $\pm$ 2.0 &  1.7 $\pm$ 1.0 &  80.4 $\pm$ 20.0 &  16.0 $\pm$ 11.0 &  36.5 $\pm$ 15.0 \\
Facial-rec dist &  40.3 $\pm$ 9.0 &  13.8 $\pm$ 4.0 &  6.9 $\pm$ 2.0 &  2.3 $\pm$ 1.0 &   7.8 $\pm$ 13.0 &    1.0 $\pm$ 3.0 &    8.2 $\pm$ 9.0 \\
Combined        &  38.9 $\pm$ 8.0 &  10.2 $\pm$ 3.0 &  6.5 $\pm$ 2.0 &  2.0 $\pm$ 1.0 &  36.0 $\pm$ 24.0 &    0.9 $\pm$ 3.0 &  14.8 $\pm$ 11.0 \\
\bottomrule
\end{tabular}



# calculate metrics for matches
(can skip this since it's already precomputed)

In [ ]:
dists_gan = data.get_dists('gan')
dists_facial = data.get_dists('facial')
# dists_facial = data.get_dists('facenet_vgg2')
dists_vgg = data.get_dists('vgg')

def select_dist(dists_match_name):
    if dists_match_name == 'facial':
        return dists_facial
    elif dists_match_name == 'vgg':
        return dists_vgg
    elif dists_match_name == 'gan':
        return dists_gan
    elif dists_match_name == 'gan_constrained':
        return dists_gan + (dists_facial > 0.6) * 1e3 # constraint for missclassificaiton

r = {
    k: [] for k in all_keys + [kk + '_std' for kk in all_keys]
}

for dists_match_name in dists_match_names:
    dists_match = select_dist(dists_match_name)
    lists = {
        k: [] for k in all_keys
    }
    print('calculating', dists_match_name)
    for im_idx in tqdm(range(N_IMS)):
        orig = d.iloc[im_idx]
        id_orig = orig.id

        # id retention
        dists_im = dists_match[im_idx]
        # these indexes are all in df.space (not d.space)
        matched_idxs = np.argsort(dists_im)
        matched_ids = df.iloc[matched_idxs].id.values
        matched_idxs_diff = matched_idxs[matched_ids[matched_ids != id_orig]]
        
        # to convert back to d, we get the index from df
        matched_df_index = df.iloc[matched_idxs].index.values
        matched_df_index_diff = df.iloc[matched_idxs_diff].index.values
        
        # preserving id
        lists['err_top1'].append(id_orig not in matched_ids[:1])
        lists['err_top5'].append(id_orig not in matched_ids[:5])
        lists['err_top10'].append(id_orig not in matched_ids[:10])
        
        # 2 types of matching
        # d_full = df.iloc[matched_idxs[:10]]
        d_full = df.loc[matched_df_index[:10]]
        d_diff = df.loc[matched_df_index_diff[:10]] # df.iloc[matched_idxs_diff[:10]]
        
        for dd, suff in zip([d_full, d_diff], suffs):
            # binary feats
            for k in attr_keys:
                lists[k + suff].append(np.mean(dd[k] != orig[k]))
            
            # numerical feats
            for k in numerical_keys:
                lists[k + suff].append(np.mean(np.abs(dd[k] - orig[k])))
    
    # condense into one value per key/dist
    for k in lists.keys():
        r[k].append(np.mean(lists[k]))
        r[k + '_std'].append(np.std(lists[k]))
            
r = pd.DataFrame.from_dict(r)
r.N_IMS = N_IMS
r.index = dists_match_names
r.to_pickle(oj(DIR_PROCESSED, 'matches_top10_stats.pkl'))